In [10]:
import os
from datasets import load_dataset
from huggingface_hub import login, create_repo

# Load environment variables from a .env file (root at ../.env, fallback to local)
def load_env_file(path: str) -> None:
    if not os.path.exists(path):
        return
    try:
        with open(path, "r", encoding="utf-8") as env_file:
            for raw_line in env_file:
                line = raw_line.strip()
                if not line or line.startswith("#") or "=" not in line:
                    continue
                key, value = line.split("=", 1)
                key = key.strip()
                value = value.strip().strip('"').strip("'")
                if key and key not in os.environ:
                    os.environ[key] = value
    except Exception:
        pass

# First try root .env at project root, then a local .env next to the notebook
load_env_file("../.env")
load_env_file(".env")

# Authenticate with Hugging Face using an env var token (HF_TOKEN or HUGGINGFACE_HUB_TOKEN)
hf_token = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_HUB_TOKEN")
if not hf_token:
    raise RuntimeError(
        "Missing Hugging Face token. Put HF_TOKEN in the repo root .env or set it in your environment to push to the Hub."
    )
login(token=hf_token)

# Ensure the target dataset repo exists
repo_id = "raxITLabs/GrayZone"
create_repo(repo_id, repo_type="dataset", exist_ok=True, token=hf_token)

# Step 1 — Load HarmBench datasets (no push here)
print("Loading HarmBench datasets ...")
ds_standard = load_dataset("walledai/HarmBench", "standard")
ds_contextual = load_dataset("walledai/HarmBench", "contextual")

# Keep original schemas (no added columns)
standard_train = ds_standard["train"]
contextual_train = ds_contextual["train"]

# Keep splits for CSV export/import steps
print("Prepared splits (original schemas):")
print(" - standard:", len(standard_train), standard_train.column_names)
print(" - contextual:", len(contextual_train), contextual_train.column_names)

# Where CSVs will be written/read (next cells use this path)
csv_dir = "./grayzone_csv"
os.makedirs(csv_dir, exist_ok=True)
print("CSV directory:", os.path.abspath(csv_dir))


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Loading HarmBench datasets ...
Prepared splits (original schemas):
 - standard: 200 ['prompt', 'category']
 - contextual: 100 ['prompt', 'context', 'category']
CSV directory: /Users/adeshgairola/Documents/Test-temp-folder/openai-safety-bench/dataset/grayzone_csv


In [11]:
# Step 2 — Filter category == 'cybercrime_intrusion' and export to CSV
# Writes filtered CSVs under csv_dir

import pandas as pd

# Filter both splits by category
standard_filtered = standard_train.filter(lambda ex: ex["category"] == "cybercrime_intrusion")
contextual_filtered = contextual_train.filter(lambda ex: ex["category"] == "cybercrime_intrusion")

# Output paths
standard_csv_path = os.path.join(csv_dir, "standard.cybercrime_intrusion.csv")
contextual_csv_path = os.path.join(csv_dir, "contextual.cybercrime_intrusion.csv")

# Export to CSV
standard_filtered.to_pandas().to_csv(standard_csv_path, index=False)
contextual_filtered.to_pandas().to_csv(contextual_csv_path, index=False)

print("Filtered counts:")
print(" - standard:", len(standard_filtered))
print(" - contextual:", len(contextual_filtered))
print("Wrote:")
print(" -", os.path.abspath(standard_csv_path))
print(" -", os.path.abspath(contextual_csv_path))


Filter: 100%|██████████| 100/100 [00:00<00:00, 56817.99 examples/s]

Filtered counts:
 - standard: 40
 - contextual: 27
Wrote:
 - /Users/adeshgairola/Documents/Test-temp-folder/openai-safety-bench/dataset/grayzone_csv/standard.cybercrime_intrusion.csv
 - /Users/adeshgairola/Documents/Test-temp-folder/openai-safety-bench/dataset/grayzone_csv/contextual.cybercrime_intrusion.csv


In [14]:
# Step 3 — Re-import edited CSVs preserving each split's original schema
# Edit the CSVs in `csv_dir` manually, then run this cell to re-import.

import pandas as pd
from datasets import Dataset, DatasetDict

# Read edited CSVs
standard_csv_path = os.path.join(csv_dir, "standard.cybercrime_intrusion.csv")
contextual_csv_path = os.path.join(csv_dir, "contextual.cybercrime_intrusion.csv")

standard_df = pd.read_csv(standard_csv_path)
contextual_df = pd.read_csv(contextual_csv_path)

# Do not add or reorder columns — keep as-is for each split
standard_ds = Dataset.from_pandas(standard_df, preserve_index=False)
contextual_ds = Dataset.from_pandas(contextual_df, preserve_index=False)

print("standard columns:", standard_ds.column_names)
print("contextual columns:", contextual_ds.column_names)

# Build two separate DatasetDicts (one per config) since schemas differ
combined_standard = DatasetDict({"train": standard_ds})
combined_contextual = DatasetDict({"train": contextual_ds})

print("Ready to push two configs: standard (train) and contextual (train)")

standard columns: ['prompt', 'category']
contextual columns: ['prompt', 'context', 'category']
Ready to push two configs: standard (train) and contextual (train)


In [15]:
# Step 4 — Simple push: direct to org main; on 403, fall back to your user repo (no PR/branches)
from huggingface_hub import HfApi
from huggingface_hub.utils import HfHubHTTPError

api = HfApi()

# Detect current user for fallback
who = api.whoami(token=hf_token)
user_name = who.get("name") or who.get("email") or "user"

try:
    print(f"Pushing both configs directly to {repo_id} main ...")
    combined_standard.push_to_hub(
        repo_id,
        config_name="standard",
        token=hf_token,
        private=False,
        revision="main",
        create_pr=False,
        commit_message="Upload GrayZone standard (train) to main",
    )
    combined_contextual.push_to_hub(
        repo_id,
        config_name="contextual",
        token=hf_token,
        private=False,
        revision="main",
        create_pr=False,
        commit_message="Upload GrayZone contextual (train) to main",
    )
    pushed_repo = repo_id
    print("Pushed directly to main.")
except HfHubHTTPError as e:
    print(f"Direct push to org main failed with {e}. Falling back to user namespace.")
    fallback_repo_id = f"{user_name}/GrayZone"
    create_repo(fallback_repo_id, repo_type="dataset", exist_ok=True, token=hf_token)

    combined_standard.push_to_hub(
        fallback_repo_id,
        config_name="standard",
        token=hf_token,
        private=False,
        revision="main",
        create_pr=False,
        commit_message="Upload GrayZone standard (train) to user main",
    )
    combined_contextual.push_to_hub(
        fallback_repo_id,
        config_name="contextual",
        token=hf_token,
        private=False,
        revision="main",
        create_pr=False,
        commit_message="Upload GrayZone contextual (train) to user main",
    )
    pushed_repo = fallback_repo_id
    print("Pushed to user repo main.")

print("Done. Check:", f"https://huggingface.co/datasets/{pushed_repo}")


Pushing both configs directly to raxITLabs/GrayZone main ...


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 327.02ba/s]
Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

Processing Files (1 / 1)                : 100%|██████████| 3.55kB / 3.55kB, 2.96kB/s  


Processing Files (1 / 1)                : 100%|██████████| 3.55kB / 3.55kB, 2.22kB/s  
New Data Upload                         : 100%|██████████| 3.55kB / 3.55kB, 2.22kB/s  
                                        : 100%|██████████| 3.55kB / 3.55kB            
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 1133.90ba/s]
Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

Processing Files (1 / 1)                : 100%|██████████| 24.1kB / 24.1kB, 20.0kB/s  


Processing Files (1 / 1)                : 100%|██████████| 24.1kB / 24.1kB, 15.0kB/s  
New Data Upload                         : 100%|██████████| 24.1kB / 24.1kB, 15.0kB/s  
                                        : 100%|█████

Pushed directly to main.
Done. Check: https://huggingface.co/datasets/raxITLabs/GrayZone
